In [1]:
import os
import pandas as pd
from lxml import html
import requests
import re
import sys
from datetime import datetime

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import time
import pickle
import string

# Input the CIK list for the fund family you are looking at

In [2]:
fund_name = 'morgan_stanley'


CIK_LIST = ['741375', '1227155', '315812', '356409', '832705', '1161973', '836487', '730044', '716716', '860720', '93284', '806564', '916618', '93285', '924394', '1388141', '878929', '1368493', '904112', '859037', '919808', '876162', '1072552', '1132218', '1002427', '882381', '314366', '882381', '1074111', '889128']




In [3]:
cur_wd = os.getcwd()
if 'trunk' in cur_wd:
    output_directory = os.path.join("\\".join(cur_wd.split('\\')[0:-1]),'output')
else:
    output_directory = os.path.join("\\".join(cur_wd.split('\\')[0:-1]),'output','working')
    
cached_wd = os.path.join("\\".join(cur_wd.split('\\')[0:-1]),'cached_data')
cur_wd = os.path.join("\\".join(cur_wd.split('\\')[0:-1]),'data')
print('data directory')
print(cur_wd)
print('output directory')
print(output_directory)

print('cached data')
print(cached_wd)


data directory
C:\Users\jjanko\Desktop\wei_mutual_fund_project\data
output directory
C:\Users\jjanko\Desktop\wei_mutual_fund_project\output\working
cached data
C:\Users\jjanko\Desktop\wei_mutual_fund_project\cached_data


# Helper Functions for Analysis

In [4]:
def get_cleaned(lines):
    cleaned = []
    for i in lines:
        #gets rid of tags
        cleaned_i = re.sub('<[^>]*>', ' ', i)
        #gets rid of &nbsp; and replaces with space
        cleaned_i = re.sub('&nbsp;', ' ', cleaned_i)
        #re.sub("<.*?>","",st)
        if not re.match(r'^\s*$', cleaned_i):
            cleaned.append(cleaned_i)
    return cleaned

def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]


def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString))


def remove_junk(addline):
    addline = re.sub('<[^>]*>', ' ', addline)
    addline = re.sub('&nbsp;', ' ', addline)
    addine = re.sub('&#174;', ' ', addline)
    addline = re.sub('&#8480;', ' ', addline)
    addline = re.sub(r'&reg;',' ',addline)
    addline = re.sub(r'\n', ' ', addline)
    addline = re.sub(r'\t', ' ', addline)
    return addline

def getDuplicateColumns(df): 
  
    # Create an empty set 
    duplicateColumnNames = set() 
      
    # Iterate through all the columns  
    # of dataframe 
    for x in range(df.shape[1]): 
          
        # Take column at xth index. 
        col = df.iloc[:, x] 
          
        # Iterate through all the columns in 
        # DataFrame from (x + 1)th index to 
        # last index 
        for y in range(x + 1, df.shape[1]): 
              
            # Take column at yth index. 
            otherCol = df.iloc[:, y] 
              
            # Check if two columns at x & y 
            # index are equal or not, 
            # if equal then adding  
            # to the set 
            if col.equals(otherCol): 
                duplicateColumnNames.add(df.columns.values[y]) 
                  
    # Return list of unique column names  
    # whose contents are duplicates. 
    return list(duplicateColumnNames) 

In [5]:
with open(os.path.join(cached_wd, 'errors_dict.pickle'), 'rb') as handle:
    errors = pickle.load(handle)
    
print(errors)

        

{'867297': ['0001193125-04-202597.txt', '0001193125-05-115532.txt', '0001193125-05-232633.txt'], '1091439': ['0000950152-04-008622.txt', '0000950152-05-004867.txt', '0000950152-05-009599.txt'], '1301708': ['0000950152-05-009600.txt'], '1423227': [], '1098605': ['0000950136-05-006141.txt', '0000950136-04-003174.txt', '0000950136-05-001871.txt'], '1378240': ['0001104659-07-029038.txt', '0001104659-07-077264.txt', '0001104659-08-027201.txt', '0001104659-08-064591.txt'], '726735': ['0000950129-04-009378.txt', '0000950129-05-005810.txt', '0000950129-05-011455.txt'], '844779': ['0001193125-05-039836.txt', '0001193125-05-175146.txt', '0001193125-05-175147.txt', '0001193125-05-175620.txt'], '1398078': [], '774013': ['0001193805-05-000319.txt', '0001193805-05-001781.txt', '0001193805-05-002628.txt', '0001193805-05-002636.txt', '0001193805-05-002638.txt'], '1062806': ['0000900092-04-000310.txt', '0000900092-05-000103.txt', '0000900092-05-000135.txt', '0000900092-05-000453.txt', '0001193805-04-00

In [6]:
weblink = {}
txt = []

for CIK in CIK_LIST:
    weblink[CIK]  = []
    txt_files = os.listdir(os.path.join(cur_wd, CIK, 'n-q'))
    for i in txt_files:
        txt.append(i)
        weblink[CIK].append(r"https://www.sec.gov/Archives/edgar/data/" + str(CIK)+r"/"  + i)

get panel 2

In [7]:
with open(os.path.join(cached_wd, 'panel2.pickle'), 'rb') as handle:
    panel2 = pickle.load(handle)
    


In [8]:
panel2 = panel2.loc[panel2['file_read'].isin(txt)]

In [9]:
print(panel2)
panel2.to_csv(os.path.join(output_directory, fund_name + '_panel2.csv'), sep = ',')

   CLASS-CONTRACT-ID  CLASS-CONTRACT-NAME CLASS-CONTRACT-TICKER-SYMBOL  \
0         C000011537       Class P Shares                        MFXAX   
1         C000011538       Class I Shares                        MPCXX   
2         C000011539     Investment Class                        MAFIX   
3         C000011548       Class P Shares                        MPFDX   
4         C000011549       Class I Shares                        MPLDX   
5         C000057346       Class H Shares                        MBAAX   
6         C000065127       Class L Shares                        MPBAX   
7         C000011550       Class I Shares                        MABIX   
8         C000052876       Class P Shares                        MACGX   
9         C000011555       Class P Shares                        MPEGX   
10        C000011556       Class I Shares                         None   
11        C000011557     Investment Class                         None   
12        C000011562       Class P Sha

## Test on One FILE

In [10]:

CIK  ='1002427'
text =  '0001104659-19-023942.txt'


In [11]:
names = panel2.loc[panel2['file_read'] == text, 'SERIES-NAME']
names = names.unique()
print(names)

['Morgan Stanley Insight Fund']


In [12]:
url = r'https://www.sec.gov/Archives/edgar/data/' + CIK + r'/' + text.replace('.txt', '').replace('-','')  + r'/' + text
html_table = urlopen(url).read()
soup = BeautifulSoup(html_table, "html.parser")
tags = soup.find_all(['table','p'])

In [13]:
print(url)

https://www.sec.gov/Archives/edgar/data/1002427/000110465919023942/0001104659-19-023942.txt


In [14]:
tables = []
iterate = 0
for i in tags:
    for n in names:
        if n.lower() in str(i).lower().replace('\n',' '):
            fundname = n
            print(fundname)
            
    try:
        df = pd.read_html(str(i), flavor="bs4")
        for j in df:
            if len(j.index) > 0 :
                tables.append(j)
                iterate = iterate + 1
    except:
        pass

Morgan Stanley Insight Fund
Morgan Stanley Insight Fund
Morgan Stanley Insight Fund
Morgan Stanley Insight Fund
Morgan Stanley Insight Fund


In [15]:
def get_legend(df):
    df = df.dropna(axis = 0, how = 'all')
    df = df.dropna(axis = 1, how = 'all')
    legend = pd.DataFrame(columns = ['code', 'symbol', 'restricted'])
    for count, row in df.iterrows():
        keys = str(row[df.columns[0]]).strip()
        code = str(row[df.columns[1]])
        if len(keys) == 3 and '(' in keys and ')' in keys:
            add = pd.DataFrame([code, keys, '']).T
            add.columns = ['code', 'symbol', 'restricted']
            if len(legend.index) == 0:
                legend = add.copy()
            else:
                legend = pd.concat([legend, add], axis = 0,ignore_index = True, sort = False)
    return legend

### check legend 

In [16]:
try:
    df = tables[4]
    legend = get_legend(df)
    print(legend)
except: 
    print("you are reading the wrong table")

                                                code symbol restricted
0                     Non-income producing security.    (a)           
1  All or a portion of this security was on loan....    (b)           
2  At February 28, 2019, the Fund held fair value...    (c)           
3                                 Illiquid security.    (d)           
4  Security cannot be offered for public resale w...    (e)           
5  The Fund invests in the Institutional Class of...    (f)           
6  The Fund is permitted to purchase and sell sec...    (g)           
7  At February 28, 2019, the aggregate cost for f...    (h)           


In [17]:
def check_level_columns(df):
    check_found = True
    for col in df.columns:
        check_name = df[col][df[col].apply(lambda x: re.sub(r'\s', '', str(x)).lower()).str.contains('level1')]
        if len(check_name) > 0:
            check_found = False
    return check_found

In [18]:
def get_holdings(df, headername):
    cleaned_df = pd.DataFrame()
    df = df.dropna(axis = 0, how = 'all')
    df = df.dropna(axis = 1, how = 'all')
    check_levels = check_level_columns(df)
    acq = pd.DataFrame(columns = ['acq cost', 'acq date', 'acq name'])

    if len(df.columns) >= 4 and check_levels:
        name_col = 1
        df = df.dropna(axis = 0, subset = [df.columns[name_col]])
        df = df.dropna(thresh = 3, axis = 0)
        non_column_name = df.drop(df.columns[name_col], axis = 1)
        for c in non_column_name.columns:
            if non_column_name[c].str.contains('$').any():
                non_column_name[c] = non_column_name[c].str.replace('$', '')
        non_column_name.replace("", float("NaN"), inplace=True)
        for count, row in df.iterrows():
            name = str(row[df.columns[name_col]])
            key = re.findall('\(.*?\)',name)
            acquisition_cost = ''
            acquisition_date = ''
            for k in key:
                name = name.replace(k,'').strip()
                if 'acquisition' in k:
                    acq_split = k.split(';')
                    acquisition_cost = acq_split[0].split('-')[1].strip().replace('$','').replace(',','')
                    acquisition_date = acq_split[1].strip().split(' ')[1].strip().replace(')','').replace(',','')
            if acquisition_cost != '':
                add_acq = pd.DataFrame([acquisition_cost,acquisition_date,name]).T
                add_acq.columns = ['acq cost', 'acq date', 'acq name']
                acq = acq.append(add_acq, ignore_index = True)

            key = ",".join(key)
            if key == 'nan':
                key = ''

            if name[-1] != r'%' and '%)' not in name and '(cost' not in name and 'PRINCIPAL AMOUNT' not in name:
                if ':' in name:
                    headername = name
                if name.split(' ')[0] == 'Series' or name[0].isdigit():
                    name = headername + ' ' + name
                add = pd.DataFrame(non_column_name.loc[count]).dropna()
                add = add[~add.duplicated()]
                if len(add) == 2:
                    if add.iloc[1].values[0].isnumeric() and add.iloc[0].values[0].isnumeric():
                        add = add.transpose().dropna()
                        add.columns = ['holdings shares', 'holdings value']
                        add['holdings name'] = name
                        add['key'] = key  

                        cleaned_df = pd.concat([cleaned_df, add], axis = 0, sort=False, ignore_index = True)

                        cleaned_df.index = pd.RangeIndex(len(cleaned_df.index))
        if len(cleaned_df.index) == 0:
            name_col = 2
            df = df.dropna(axis = 0, subset = [df.columns[name_col]])
            df = df.dropna(thresh = 3, axis = 0)
            non_column_name = df.drop(df.columns[name_col], axis = 1)
            for c in non_column_name.columns:
                if non_column_name[c].str.contains('$').any():
                    non_column_name[c] = non_column_name[c].str.replace('$', '')
            non_column_name.replace("", float("NaN"), inplace=True)
            for count, row in df.iterrows():
                name = str(row[df.columns[name_col]])
                key = re.findall('\(.*?\)',name)
                acquisition_cost = ''
                acquisition_date = ''
                for k in key:
                    name = name.replace(k,'').strip()
                    if 'acquisition' in k:
                        acq_split = k.split(';')
                        acquisition_cost = acq_split[0].split('-')[1].strip().replace('$','').replace(',','')
                        acquisition_date = acq_split[1].strip().split(' ')[1].strip().replace(')','').replace(',','')
                if acquisition_cost != '':
                    add_acq = pd.DataFrame([acquisition_cost,acquisition_date,name]).T
                    add_acq.columns = ['acq cost', 'acq date', 'acq name']
                    acq = acq.append(add_acq, ignore_index = True)

                key = "".join(key)
                if key == 'nan':
                    key = ''

                if name[-1] != r'%' and '%)' not in name and '(cost' not in name and 'PRINCIPAL AMOUNT' not in name:
                    if ':' in name:
                        headername = name
                    if name.split(' ')[0] == 'Series' or name[0].isdigit():
                        name = headername + ' ' + name
                    add = pd.DataFrame(non_column_name.loc[count]).dropna()
                    add = add[~add.duplicated()]
                    if len(add) == 2:
                        if add.iloc[1].values[0].isnumeric() and add.iloc[0].values[0].isnumeric():
                            add = add.transpose().dropna()
                            add.columns = ['holdings shares', 'holdings value']
                            add['holdings name'] = name
                            add['key'] = key  

                            cleaned_df = pd.concat([cleaned_df, add], axis = 0, sort=False, ignore_index = True)

                            cleaned_df.index = pd.RangeIndex(len(cleaned_df.index))

    return cleaned_df, headername, acq 
    

In [19]:
headername = ''
df,headername, df_acq = get_holdings(tables[2], headername)
print(df)
print(df_acq)

   holdings shares holdings value  \
0           189594       45396387   
1          1189110       45471566   
2           229650       45454625   
3            82075        2216025   
4            58155        7022216   
5            77181        3560360   
6           901243       11283562   
7           197427         481722   
8            35881       35880904   
9             1107        1107435   
10            4208        4208254   
11            3322        3322306   
12           87187       87187250   

                                        holdings name  \
0                                    ServiceNow, Inc.   
1                           Smartsheet, Inc., Class A   
2                              Workday, Inc., Class A   
3                                Magic Leap, Series C   
4                              Airbnb, Inc., Series D   
5                         Uber Technologies, Series G   
6                        10X Genomics, Inc., Series B   
7                        

# check holdings

In [20]:
df = tables[1]
cleaned_df = pd.DataFrame()
df = df.dropna(axis = 0, how = 'all')
df = df.dropna(axis = 1, how = 'all')
check_levels = check_level_columns(df)
acq = pd.DataFrame(columns = ['acq cost', 'acq date', 'acq name'])

if len(df.columns) >= 4 and check_levels:
    name_col = 1
    df = df.dropna(axis = 0, subset = [df.columns[name_col]])
    df = df.dropna(thresh = 3, axis = 0)
    non_column_name = df.drop(df.columns[name_col], axis = 1)
    for c in non_column_name.columns:
        if non_column_name[c].str.contains('$').any():
            non_column_name[c] = non_column_name[c].str.replace('$', '')
    non_column_name.replace("", float("NaN"), inplace=True)
    for count, row in df.iterrows():
        name = str(row[df.columns[name_col]])
        key = re.findall('\(.*?\)',name)
        acquisition_cost = ''
        acquisition_date = ''
        for k in key:
            name = name.replace(k,'').strip()
            if 'acquisition' in k:
                print(k)
                acq_split = k.split(';')
                acquisition_cost = acq_split[0].split('-')[1].strip().replace('$','').replace(',','')
                acquisition_date = acq_split[1].strip().split(' ')[1].strip().replace(')','').replace(',','')
        if acquisition_cost != '':
            add_acq = pd.DataFrame([acquisition_cost,acquisition_date,name]).T
            add_acq.columns = ['acq cost', 'acq date', 'acq name']
            acq = acq.append(add_acq, ignore_index = True)
                
        key = ",".join(key)
        if key == 'nan':
            key = ''

        if name[-1] != r'%' and '%)' not in name and '(cost' not in name and 'PRINCIPAL AMOUNT' not in name:
            if ':' in name:
                headername = name
            if name.split(' ')[0] == 'Series' or name[0].isdigit():
                name = headername + ' ' + name
            add = pd.DataFrame(non_column_name.loc[count]).dropna()
            add = add[~add.duplicated()]
            if len(add) == 2:
                if add.iloc[1].values[0].isnumeric() and add.iloc[0].values[0].isnumeric():
                    add = add.transpose().dropna()
                    add.columns = ['holdings shares', 'holdings value']
                    add['holdings name'] = name
                    add['key'] = key  

                    cleaned_df = pd.concat([cleaned_df, add], axis = 0, sort=False, ignore_index = True)

                    cleaned_df.index = pd.RangeIndex(len(cleaned_df.index))
    if len(cleaned_df.index) == 0:
        name_col = 2
        df = df.dropna(axis = 0, subset = [df.columns[name_col]])
        df = df.dropna(thresh = 3, axis = 0)
        non_column_name = df.drop(df.columns[name_col], axis = 1)
        for c in non_column_name.columns:
            if non_column_name[c].str.contains('$').any():
                non_column_name[c] = non_column_name[c].str.replace('$', '')
        non_column_name.replace("", float("NaN"), inplace=True)
        for count, row in df.iterrows():
            name = str(row[df.columns[name_col]])
            key = re.findall('\(.*?\)',name)
            acquisition_cost = ''
            acquisition_date = ''
            for k in key:
                name = name.replace(k,'').strip()
                if 'acquisition' in k:
                    print(k)
                    acq_split = k.split(';')
                    acquisition_cost = acq_split[0].split('-')[1].strip().replace('$','').replace(',','')
                    acquisition_date = acq_split[1].strip().split(' ')[1].strip().replace(')','').replace(',','')
            if acquisition_cost != '':
                add_acq = pd.DataFrame([acquisition_cost,acquisition_date,name]).T
                add_acq.columns = ['acq cost', 'acq date', 'acq name']
                acq = acq.append(add_acq, ignore_index = True)

            key = ",".join(key)
            if key == 'nan':
                key = ''

            if name[-1] != r'%' and '%)' not in name and '(cost' not in name and 'PRINCIPAL AMOUNT' not in name:
                if ':' in name:
                    headername = name
                if name.split(' ')[0] == 'Series' or name[0].isdigit():
                    name = headername + ' ' + name
                add = pd.DataFrame(non_column_name.loc[count]).dropna()
                add = add[~add.duplicated()]
                if len(add) == 2:
                    if add.iloc[1].values[0].isnumeric() and add.iloc[0].values[0].isnumeric():
                        add = add.transpose().dropna()
                        add.columns = ['holdings shares', 'holdings value']
                        add['holdings name'] = name
                        add['key'] = key  

                        cleaned_df = pd.concat([cleaned_df, add], axis = 0, sort=False, ignore_index = True)

                        cleaned_df.index = pd.RangeIndex(len(cleaned_df.index))
    


print(cleaned_df)
print(acq)

(acquisition cost - $3,870,089; acquired 11/07/18)
   holdings shares holdings value                  holdings name  \
0            64853        5512505  Alnylam Pharmaceuticals, Inc.   
1             7643        1186347             Bluebird Bio, Inc.   
2            49412        1019370          Editas Medicine, Inc.   
3            64925         988808    Intellia Therapeutics, Inc.   
4           108720         866498                 Intrexon Corp.   
5           173369        3918139                  Moderna, Inc.   
6           121197       43400646                  Netflix, Inc.   
7           424360       59469810          Spotify Technology SA   
8           286190       39874853                   DexCom, Inc.   
9           112245       61466484       Intuitive Surgical, Inc.   
10          123455       11506006                   LivaNova PLC   
11          116298       15545554                 Penumbra, Inc.   
12          638602       22849180                 Covetrus, Inc. 

# Run both legend and holdings scraper on test file

In [21]:
headername = ''
tables = []
holdings = pd.DataFrame()
legend = pd.DataFrame()
acq  = pd.DataFrame()
fundname = ''
identifiercount = 0
count = 0

for i in tags:
    for n in names:
        if n.lower() in str(i).lower().replace('\n',' '):
            fundname = n
                
    try:
        if fundname != '':
            df = pd.read_html(str(i), flavor="bs4")
            for t in df:
                count = count +1
                tables.append(t)
                add_legend = get_legend(t)
                if len(add_legend.index) > 0:
                    add_legend.loc[:, 'identifier'] = fundname
                    if len(legend.index) > 0:
                        legend = pd.concat([legend, add_legend], axis = 0 , sort=False, ignore_index = True)
                    else:
                        legend  = add_legend.copy()

                cleaned_df, headername, df_acq  = get_holdings(t, headername)
                
                if len(cleaned_df.index) > 1:
                    cleaned_df.loc[:, 'fund name'] = fundname
                    cleaned_df.loc[:, 'identifier'] = fundname
                    if len(holdings.index) > 0:
                        holdings = pd.concat([holdings, cleaned_df], axis = 0, sort=False, ignore_index = True)
                    else:
                        holdings = cleaned_df.copy()
                        
                if len(df_acq.index) > 1:
                    df_acq.loc[:, 'fund name'] = fundname
                    df_acq.loc[:, 'identifier'] = fundname
                    if len(acq.index) > 0:
                        acq = pd.concat([acq, df_acq], axis = 0, sort=False, ignore_index = True)
                    else:
                        acq = df_acq.copy()
    except: 
        pass
holdings = pd.concat([acq, holdings], axis = 0, sort = False, ignore_index = True)

In [22]:
holdings['value multiplier'] = ''
holdings['textfile'] = text
holdings['CIK'] = CIK
holdings['date_filed'] = '' 
holdings['company conformed name'] = ''
holdings['reporting_date'] = ''

try:
    date_filed = panel2.loc[panel2['file_read'] == text, 'date_filed'].unique()[0]
    conformed_name = panel2.loc[panel2['file_read'] == text, 'company conformed name'].unique()[0]
    reporting_date = panel2.loc[panel2['file_read'] == text, 'reporting_date'].unique()[0]
except IndexError:
    date_filed = ''
    conformed_name = ''

holdings.loc[:, 'date_filed'] = date_filed
holdings.loc[:, 'company conformed name'] = conformed_name
holdings.loc[:, 'reporting_date'] = reporting_date

holdings.index = pd.RangeIndex(len(holdings.index))
print(holdings)

for z in weblink[CIK]:
    if text in z:
        matching_link = z

holdings['weblink'] = matching_link
legend['weblink'] = matching_link
legend['textfile'] = text

   acq cost  acq date                      acq name  \
0   1890434  12/22/15          Magic Leap, Series C   
1   2367666  04/16/14        Airbnb, Inc., Series D   
2   3764289  12/03/15   Uber Technologies, Series G   
3   2947065  12/19/14  10X Genomics, Inc., Series B   
4   2255228  06/17/14       Lookout, Inc., Series F   
5       NaN       NaN                           NaN   
6       NaN       NaN                           NaN   
7       NaN       NaN                           NaN   
8       NaN       NaN                           NaN   
9       NaN       NaN                           NaN   
10      NaN       NaN                           NaN   
11      NaN       NaN                           NaN   
12      NaN       NaN                           NaN   
13      NaN       NaN                           NaN   
14      NaN       NaN                           NaN   
15      NaN       NaN                           NaN   
16      NaN       NaN                           NaN   
17      Na

In [23]:
print(holdings)
print(legend)

   acq cost  acq date                      acq name  \
0   1890434  12/22/15          Magic Leap, Series C   
1   2367666  04/16/14        Airbnb, Inc., Series D   
2   3764289  12/03/15   Uber Technologies, Series G   
3   2947065  12/19/14  10X Genomics, Inc., Series B   
4   2255228  06/17/14       Lookout, Inc., Series F   
5       NaN       NaN                           NaN   
6       NaN       NaN                           NaN   
7       NaN       NaN                           NaN   
8       NaN       NaN                           NaN   
9       NaN       NaN                           NaN   
10      NaN       NaN                           NaN   
11      NaN       NaN                           NaN   
12      NaN       NaN                           NaN   
13      NaN       NaN                           NaN   
14      NaN       NaN                           NaN   
15      NaN       NaN                           NaN   
16      NaN       NaN                           NaN   
17      Na

In [24]:
legend.loc[:,'restricted'] = 'no'

legend.loc[legend['code'].str.contains("restrict") | legend['code'].str.contains("level 3 security") \
              | legend['code'].str.contains("exempt from registration") |legend['code'].str.contains("private placement") , 'restricted'] = 'yes'

restricted = legend.loc[legend['restricted'].str.contains("yes")]
print(restricted)

holdings.loc[:,'restricted'] = 'no'



                                                code symbol restricted  \
4  Security cannot be offered for public resale w...    (e)        yes   

                    identifier  \
4  Morgan Stanley Insight Fund   

                                             weblink                  textfile  
4  https://www.sec.gov/Archives/edgar/data/100242...  0001104659-19-023942.txt  


In [381]:
#get the restricted legend and restricted securities

In [25]:
legend.loc[:,'restricted'] = 'no'

legend.loc[legend['code'].str.contains("restrict") | legend['code'].str.contains("level 3 security") \
              | legend['code'].str.contains("exempt from registration") | legend['code'].str.contains("private placement transaction"), 'restricted'] = 'yes'

restricted = legend.loc[legend['restricted'].str.contains("yes")]
print(restricted)

holdings.loc[:,'restricted'] = 'no'

for index, row in restricted.iterrows():
    holdings.loc[(holdings['identifier'] == row['identifier']) & (holdings['textfile'] == row.textfile) \
                   & (holdings['key'].str.contains(row['symbol'])), 'restricted'] = 'yes'
    
restricted_holdings = holdings.loc[(holdings['restricted'] == 'yes')]
print(restricted_holdings)

                                                code symbol restricted  \
4  Security cannot be offered for public resale w...    (e)        yes   

                    identifier  \
4  Morgan Stanley Insight Fund   

                                             weblink                  textfile  
4  https://www.sec.gov/Archives/edgar/data/100242...  0001104659-19-023942.txt  
   acq cost acq date acq name                    fund name  \
20      NaN      NaN      NaN  Morgan Stanley Insight Fund   
42      NaN      NaN      NaN  Morgan Stanley Insight Fund   
43      NaN      NaN      NaN  Morgan Stanley Insight Fund   
44      NaN      NaN      NaN  Morgan Stanley Insight Fund   
45      NaN      NaN      NaN  Morgan Stanley Insight Fund   
46      NaN      NaN      NaN  Morgan Stanley Insight Fund   
48      NaN      NaN      NaN  Morgan Stanley Insight Fund   
49      NaN      NaN      NaN  Morgan Stanley Insight Fund   
50      NaN      NaN      NaN  Morgan Stanley Insight Fund   


C:\Users\jjanko\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  del sys.path[0]


In [63]:
#get the restricted holdings

In [45]:
def getpanel1(CIK, text, panel2):
    try:
        url = r'https://www.sec.gov/Archives/edgar/data/' + CIK + r'/' + text.replace('.txt', '').replace('-','')  + r'/' + text
        print(url)
        html_table = urlopen(url).read()
        soup = BeautifulSoup(html_table, "html.parser")
        tags = soup.find_all(['table', 'p'])
        names = panel2.loc[panel2['file_read'] == text, 'SERIES-NAME']
        names = names.unique()

        headername = ''
        tables = []
        holdings = pd.DataFrame()
        legend = pd.DataFrame()
        acq  = pd.DataFrame()
        fundname = ''
        identifiercount = 0
        count = 0

        for i in tags:
            for n in names:
                if n.lower() in str(i).lower().replace('\n',' '):
                    fundname = n

            try:
                if fundname != '':
                    df = pd.read_html(str(i), flavor="bs4")
                    for t in df:
                        count = count +1
                        tables.append(t)
                        add_legend = get_legend(t)
                        if len(add_legend.index) > 0:
                            add_legend.loc[:, 'identifier'] = fundname
                            if len(legend.index) > 0:
                                legend = pd.concat([legend, add_legend], axis = 0 , sort=False, ignore_index = True)
                            else:
                                legend  = add_legend.copy()

                        cleaned_df, headername, df_acq  = get_holdings(t, headername)

                        if len(cleaned_df.index) > 1:
                            cleaned_df.loc[:, 'fund name'] = fundname
                            cleaned_df.loc[:, 'identifier'] = fundname
                            if len(holdings.index) > 0:
                                holdings = pd.concat([holdings, cleaned_df], axis = 0, sort=False, ignore_index = True)
                            else:
                                holdings = cleaned_df.copy()

                        if len(df_acq.index) > 1:
                            df_acq.loc[:, 'fund name'] = fundname
                            df_acq.loc[:, 'identifier'] = fundname
                            if len(acq.index) > 0:
                                acq = pd.concat([acq, df_acq], axis = 0, sort=False, ignore_index = True)
                            else:
                                acq = df_acq.copy()
            except: 
                pass

        holdings = pd.concat([acq, holdings], axis = 0, sort = False, ignore_index = True)

        holdings['value multiplier'] = ''
        holdings['textfile'] = text
        holdings['CIK'] = CIK
        holdings['date_filed'] = '' 
        holdings['company conformed name'] = ''
        holdings['reporting_date'] = ''

        try:
            date_filed = panel2.loc[panel2['file_read'] == text, 'date_filed'].unique()[0]
            conformed_name = panel2.loc[panel2['file_read'] == text, 'company conformed name'].unique()[0]
            reporting_date = panel2.loc[panel2['file_read'] == text, 'reporting_date'].unique()[0]
        except IndexError:
            date_filed = ''
            conformed_name = ''

        holdings.loc[:, 'date_filed'] = date_filed
        holdings.loc[:, 'company conformed name'] = conformed_name
        holdings.loc[:, 'reporting_date'] = reporting_date

        holdings.index = pd.RangeIndex(len(holdings.index))

        for z in weblink[CIK]:
            if text in z:
                matching_link = z

        holdings['weblink'] = matching_link
        legend['weblink'] = matching_link
        legend['textfile'] = text
        
    except Exception as e:
        print(e)
        holdings = pd.DataFrame()
        legend = pd.DataFrame()
        
    return holdings, legend




check panel1 for the sample file

In [34]:
panel2

,CLASS-CONTRACT-ID,CLASS-CONTRACT-NAME,CLASS-CONTRACT-TICKER-SYMBOL,SERIES-ID,OWNER-CIK,SERIES-NAME,file_read,date_filed,company conformed name,reporting_date
0,C000011537,Class P Shares,MFXAX,S000004114,0000741375,Core Plus Fixed Income Portfolio,0000950123-11-020578.txt,20110301,MORGAN STANLEY INSTITUTIONAL FUND TRUST,20101231
1,C000011538,Class I Shares,MPCXX,S000004114,0000741375,Core Plus Fixed Income Portfolio,0000950123-11-020578.txt,20110301,MORGAN STANLEY INSTITUTIONAL FUND TRUST,20101231
2,C000011539,Investment Class,MAFIX,S000004114,0000741375,Core Plus Fixed Income Portfolio,0000950123-11-020578.txt,20110301,MORGAN STANLEY INSTITUTIONAL FUND TRUST,20101231
3,C000011548,Class P Shares,MPFDX,S000004118,0000741375,Investment Grade Fixed Income Portfolio,0000950123-11-020578.txt,20110301,MORGAN STANLEY INSTITUTIONAL FUND TRUST,20101231
4,C000011549,Class I Shares,MPLDX,S000004118,0000741375,Investment Grade Fixed Income Portfolio,0000950123-11-020578.txt,20110301,MORGAN STANLEY INSTITUTIONAL FUND TRUST,20101231
5,C000057346,Class H Shares,MBAAX,S000004118,0000741375,Investment Grade Fixed Income Portfolio,0000950123-11-020578.txt,20110301,MORGAN STANLEY INSTITUTIONAL FUND TRUST,20101231
6,C000065127,Class L Shares,MPBAX,S000004118,0000741375,Investment Grade Fixed Income Portfolio,0000950123-11-020578.txt,20110301,MORGAN STANLEY INSTITUTIONAL FUND TRUST,20101231
7,C000011550,Class I Shares,MABIX,S000004119,0000741375,Limited Duration Portfolio,0000950123-11-020578.txt,20110301,MORGAN STANLEY INSTITUTIONAL FUND TRUST,20101231
8,C000052876,Class P Shares,MACGX,S000004119,0000741375,Limited Duration Portfolio,0000950123-11-020578.txt,20110301,MORGAN STANLEY INSTITUTIONAL FUND TRUST,20101231
9,C000011555,Class P Shares,MPEGX,S000004124,0000741375,Balanced Portfolio,0000950123-11-020578.txt,20110301,MORGAN STANLEY INSTITUTIONAL FUND TRUST,20101231


In [33]:

holdings, legend = getpanel1('1002427', '0000950123-04-012694.txt', panel2)

https://www.sec.gov/Archives/edgar/data/1002427/000095012304012694/0000950123-04-012694.txt


UnboundLocalError: local variable 'reporting_date' referenced before assignment

In [28]:
holdings.to_csv(os.path.join(output_directory, fund_name + '_test_holdings2.csv'), sep = ',')
legend.to_csv(os.path.join(output_directory, fund_name + '_test_legend2.csv'), sep = ',')
print(holdings)

   acq cost  acq date                      acq name  \
0   1890434  12/22/15          Magic Leap, Series C   
1   2367666  04/16/14        Airbnb, Inc., Series D   
2   3764289  12/03/15   Uber Technologies, Series G   
3   2947065  12/19/14  10X Genomics, Inc., Series B   
4   2255228  06/17/14       Lookout, Inc., Series F   
5       NaN       NaN                           NaN   
6       NaN       NaN                           NaN   
7       NaN       NaN                           NaN   
8       NaN       NaN                           NaN   
9       NaN       NaN                           NaN   
10      NaN       NaN                           NaN   
11      NaN       NaN                           NaN   
12      NaN       NaN                           NaN   
13      NaN       NaN                           NaN   
14      NaN       NaN                           NaN   
15      NaN       NaN                           NaN   
16      NaN       NaN                           NaN   
17      Na

## Run on All CIKS

In [46]:
panel1 = pd.DataFrame()


panel1 = pd.DataFrame()
panel1_legend = pd.DataFrame()
error_panel1 = []
no_holdings = {}

#CIK_LIST = ['1002427']
CIK_LIST = ['741375', '1227155', '315812', '356409', '832705', '1161973', '836487', '730044', '716716', '860720', '93284', '806564', '916618', '93285', '924394', '1388141', '878929', '1368493', '904112', '859037', '919808', '876162', '1072552', '1132218', '1002427', '882381', '314366', '882381', '1074111', '889128']


for CIK in CIK_LIST:
    print(CIK)
    no_holdings[CIK] = []
    txt_files = os.listdir(os.path.join(cur_wd, CIK, 'n-q'))
    for i in txt_files:
        if '.csv' not in i and i not in errors[CIK]:
            url = r'https://www.sec.gov/Archives/edgar/data/' + CIK + r'/' + text.replace('.txt', '').replace('-','')  + r'/' + i
            holdings, legend = getpanel1(CIK, i, panel2)
            if len(holdings) == 0:
                no_holdings[CIK].append(i)

            if panel1.empty:
                panel1 = holdings.copy()
                panel1_legend = legend.copy()
            else:
                panel1 = pd.concat([panel1, holdings], axis = 0, ignore_index = True, sort = False)
                panel1_legend = pd.concat([panel1_legend , legend], axis = 0, ignore_index = True, sort = False)

            panel1.index = pd.RangeIndex(len(panel1.index))


741375
https://www.sec.gov/Archives/edgar/data/741375/000095012311020578/0000950123-11-020578.txt
https://www.sec.gov/Archives/edgar/data/741375/000110465906013106/0001104659-06-013106.txt
https://www.sec.gov/Archives/edgar/data/741375/000110465906058008/0001104659-06-058008.txt
https://www.sec.gov/Archives/edgar/data/741375/000110465907015562/0001104659-07-015562.txt
https://www.sec.gov/Archives/edgar/data/741375/000110465907041824/0001104659-07-041824.txt
https://www.sec.gov/Archives/edgar/data/741375/000110465907065844/0001104659-07-065844.txt
https://www.sec.gov/Archives/edgar/data/741375/000110465908013402/0001104659-08-013402.txt
https://www.sec.gov/Archives/edgar/data/741375/000110465908056108/0001104659-08-056108.txt
https://www.sec.gov/Archives/edgar/data/741375/000110465909013405/0001104659-09-013405.txt
https://www.sec.gov/Archives/edgar/data/741375/000110465909052285/0001104659-09-052285.txt
https://www.sec.gov/Archives/edgar/data/741375/000110465910010046/0001104659-10-010

https://www.sec.gov/Archives/edgar/data/356409/000110465915048329/0001104659-15-048329.txt
https://www.sec.gov/Archives/edgar/data/356409/000110465915086896/0001104659-15-086896.txt
https://www.sec.gov/Archives/edgar/data/356409/000110465916129435/0001104659-16-129435.txt
https://www.sec.gov/Archives/edgar/data/356409/000110465916162981/0001104659-16-162981.txt
https://www.sec.gov/Archives/edgar/data/356409/000110465917040946/0001104659-17-040946.txt
https://www.sec.gov/Archives/edgar/data/356409/000110465917075027/0001104659-17-075027.txt
https://www.sec.gov/Archives/edgar/data/356409/000110465918042184/0001104659-18-042184.txt
https://www.sec.gov/Archives/edgar/data/356409/000110465918074084/0001104659-18-074084.txt
https://www.sec.gov/Archives/edgar/data/356409/000110465919038125/0001104659-19-038125.txt
https://www.sec.gov/Archives/edgar/data/356409/000119312511356277/0001193125-11-356277.txt
https://www.sec.gov/Archives/edgar/data/356409/000119312512285508/0001193125-12-285508.txt

https://www.sec.gov/Archives/edgar/data/716716/000095012311100941/0000950123-11-100941.txt
https://www.sec.gov/Archives/edgar/data/716716/000110465906037813/0001104659-06-037813.txt
https://www.sec.gov/Archives/edgar/data/716716/000110465906078470/0001104659-06-078470.txt
https://www.sec.gov/Archives/edgar/data/716716/000110465907044059/0001104659-07-044059.txt
https://www.sec.gov/Archives/edgar/data/716716/000110465907086034/0001104659-07-086034.txt
https://www.sec.gov/Archives/edgar/data/716716/000110465908036524/0001104659-08-036524.txt
https://www.sec.gov/Archives/edgar/data/716716/000110465908072946/0001104659-08-072946.txt
https://www.sec.gov/Archives/edgar/data/716716/000110465909035377/0001104659-09-035377.txt
https://www.sec.gov/Archives/edgar/data/716716/000110465909067650/0001104659-09-067650.txt
https://www.sec.gov/Archives/edgar/data/716716/000110465910031450/0001104659-10-031450.txt
https://www.sec.gov/Archives/edgar/data/716716/000110465910060318/0001104659-10-060318.txt

https://www.sec.gov/Archives/edgar/data/93285/000110465911003861/0001104659-11-003861.txt
https://www.sec.gov/Archives/edgar/data/93285/000110465915004662/0001104659-15-004662.txt
https://www.sec.gov/Archives/edgar/data/93285/000110465915052792/0001104659-15-052792.txt
https://www.sec.gov/Archives/edgar/data/93285/000110465916091650/0001104659-16-091650.txt
https://www.sec.gov/Archives/edgar/data/93285/000110465916133683/0001104659-16-133683.txt
https://www.sec.gov/Archives/edgar/data/93285/000110465917004206/0001104659-17-004206.txt
https://www.sec.gov/Archives/edgar/data/93285/000119312512029319/0001193125-12-029319.txt
https://www.sec.gov/Archives/edgar/data/93285/000119312512322806/0001193125-12-322806.txt
https://www.sec.gov/Archives/edgar/data/93285/000119312513057640/0001193125-13-057640.txt
https://www.sec.gov/Archives/edgar/data/93285/000119312513351673/0001193125-13-351673.txt
https://www.sec.gov/Archives/edgar/data/93285/000119312514026734/0001193125-14-026734.txt
https://ww

https://www.sec.gov/Archives/edgar/data/1072552/000110465909056676/0001104659-09-056676.txt
https://www.sec.gov/Archives/edgar/data/1072552/000110465910017764/0001104659-10-017764.txt
https://www.sec.gov/Archives/edgar/data/1072552/000110465910050221/0001104659-10-050221.txt
https://www.sec.gov/Archives/edgar/data/1072552/000119312512142691/0001193125-12-142691.txt
https://www.sec.gov/Archives/edgar/data/1072552/000119312512408330/0001193125-12-408330.txt
1132218
https://www.sec.gov/Archives/edgar/data/1132218/000095012308008509/0000950123-08-008509.txt
https://www.sec.gov/Archives/edgar/data/1132218/000095012309001396/0000950123-09-001396.txt
https://www.sec.gov/Archives/edgar/data/1132218/000095012309026752/0000950123-09-026752.txt
https://www.sec.gov/Archives/edgar/data/1132218/000095012310006654/0000950123-10-006654.txt
https://www.sec.gov/Archives/edgar/data/1132218/000095012310069598/0000950123-10-069598.txt
https://www.sec.gov/Archives/edgar/data/1132218/000095012311007121/00009

https://www.sec.gov/Archives/edgar/data/882381/000110465917058775/0001104659-17-058775.txt
https://www.sec.gov/Archives/edgar/data/882381/000110465918019772/0001104659-18-019772.txt
https://www.sec.gov/Archives/edgar/data/882381/000110465918058137/0001104659-18-058137.txt
https://www.sec.gov/Archives/edgar/data/882381/000110465919017771/0001104659-19-017771.txt
https://www.sec.gov/Archives/edgar/data/882381/000119312512142737/0001193125-12-142737.txt
https://www.sec.gov/Archives/edgar/data/882381/000119312512408421/0001193125-12-408421.txt
https://www.sec.gov/Archives/edgar/data/882381/000119312513131796/0001193125-13-131796.txt
https://www.sec.gov/Archives/edgar/data/882381/000119312513384468/0001193125-13-384468.txt
https://www.sec.gov/Archives/edgar/data/882381/000119312514123507/0001193125-14-123507.txt
https://www.sec.gov/Archives/edgar/data/882381/000119312514352774/0001193125-14-352774.txt
1074111
https://www.sec.gov/Archives/edgar/data/1074111/000095012306013009/0000950123-06-0

In [48]:
panel1.loc[:,'CIK'].unique()

array(['716716', '860720', '806564', '924394', '1072552', '1132218',
       '1002427', '882381', '314366', '1074111', '889128'], dtype=object)

In [49]:
panel1.loc[:,'acq name'] = ''
panel1.loc[:,'acq date'] = ''
panel1.loc[:,'acq cost'] = ''
print(output_directory)
panel1.to_csv(os.path.join(output_directory, fund_name + '_panel1.csv'), sep = ',')
panel1_legend.to_csv(os.path.join(output_directory, fund_name + '_panel1_legend.csv'), sep = ',')

C:\Users\jjanko\Desktop\wei_mutual_fund_project\output\working


In [50]:
panel1['CIK'].unique()

array(['716716', '860720', '806564', '924394', '1072552', '1132218',
       '1002427', '882381', '314366', '1074111', '889128'], dtype=object)

In [51]:

panel1_legend.loc[:,'restricted'] = 'no'

panel1_legend.loc[panel1_legend['code'].str.contains("restrict") | panel1_legend['code'].str.contains("level 3 security") \
              | panel1_legend['code'].str.contains("exempt from registration") |panel1_legend['code'].str.contains("private placement") , 'restricted'] = 'yes'

restricted = panel1_legend.loc[panel1_legend['restricted'].str.contains("yes")]

restricted.to_csv(os.path.join(output_directory, fund_name + '_panel1_restricted_legend.csv'), sep = ',')
print(restricted)

panel1.loc[:,'restricted'] = 'no'

for index, row in restricted.iterrows():
    try:
        panel1.loc[(panel1['identifier'] == row['identifier']) & (panel1['textfile'] == row.textfile) \
                       & (panel1['key'].str.contains(row['symbol'])), 'restricted'] = 'yes'
    except:
        pass
    
restricted_holdings = panel1.loc[(panel1['restricted'] == 'yes')]
restricted_holdings.to_csv(os.path.join(output_directory, fund_name + '_panel1_restricted.csv'), sep = ',')

                                                    code symbol restricted  \
0      Resale is restricted to qualified institutiona...    (a)        yes   
4      Resale is restricted to qualified institutiona...    (a)        yes   
8      Resale is restricted to qualified institutiona...    (a)        yes   
12     Resale is restricted to qualified institutiona...    (a)        yes   
25     Resale is restricted to qualified institutiona...    (a)        yes   
34     Resale is restricted to qualified institutiona...    (a)        yes   
82     Illiquid security. Resale is restricted to qua...    (c)        yes   
88     Illiquid security. Resale is restricted to qua...    (c)        yes   
99     Illiquid security. Resale is restricted to qua...    (c)        yes   
105    Illiquid security. Resale is restricted to qua...    (c)        yes   
116    Illiquid security. Resale is restricted to qua...    (c)        yes   
122    Illiquid security. Resale is restricted to qua...    (c) 

C:\Users\jjanko\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  app.launch_new_instance()


In [52]:
restricted_holdings

,holdings shares,holdings value,holdings name,key,fund name,identifier,value multiplier,textfile,CIK,date_filed,company conformed name,reporting_date,weblink,acq cost,acq date,acq name,restricted
0,89845,177963,Australian Infrastructure Fund,(Stapled Securities)(a)(b),EUROPEAN EQUITY PORTFOLIO,EUROPEAN EQUITY PORTFOLIO,,0000950123-11-054817.txt,716716,20110527,MORGAN STANLEY VARIABLE INVESTMENT SERIES,20110331,https://www.sec.gov/Archives/edgar/data/716716...,,,,yes
1,280513,882053,MAP Group,(Stapled Securities)(a),EUROPEAN EQUITY PORTFOLIO,EUROPEAN EQUITY PORTFOLIO,,0000950123-11-054817.txt,716716,20110527,MORGAN STANLEY VARIABLE INVESTMENT SERIES,20110331,https://www.sec.gov/Archives/edgar/data/716716...,,,,yes
2,156400,270969,DUET Group,(Stapled Securities)(a)(b),EUROPEAN EQUITY PORTFOLIO,EUROPEAN EQUITY PORTFOLIO,,0000950123-11-054817.txt,716716,20110527,MORGAN STANLEY VARIABLE INVESTMENT SERIES,20110331,https://www.sec.gov/Archives/edgar/data/716716...,,,,yes
3,105100,457670,APA Group,(Stapled Securities)(a),EUROPEAN EQUITY PORTFOLIO,EUROPEAN EQUITY PORTFOLIO,,0000950123-11-054817.txt,716716,20110527,MORGAN STANLEY VARIABLE INVESTMENT SERIES,20110331,https://www.sec.gov/Archives/edgar/data/716716...,,,,yes
4,229100,1272528,Transurban Group,(Stapled Securities)(a)(b),EUROPEAN EQUITY PORTFOLIO,EUROPEAN EQUITY PORTFOLIO,,0000950123-11-054817.txt,716716,20110527,MORGAN STANLEY VARIABLE INVESTMENT SERIES,20110331,https://www.sec.gov/Archives/edgar/data/716716...,,,,yes
9,10150,270632,Westshore Terminals Investment Corp.,(Stapled Securities)(a)(b),EUROPEAN EQUITY PORTFOLIO,EUROPEAN EQUITY PORTFOLIO,,0000950123-11-054817.txt,716716,20110527,MORGAN STANLEY VARIABLE INVESTMENT SERIES,20110331,https://www.sec.gov/Archives/edgar/data/716716...,,,,yes
11,595000,3396263,Beijing Enterprises Holdings Ltd.,(c),EUROPEAN EQUITY PORTFOLIO,EUROPEAN EQUITY PORTFOLIO,,0000950123-11-054817.txt,716716,20110527,MORGAN STANLEY VARIABLE INVESTMENT SERIES,20110331,https://www.sec.gov/Archives/edgar/data/716716...,,,,yes
12,340000,166972,China Gas Holdings Ltd.,(c),EUROPEAN EQUITY PORTFOLIO,EUROPEAN EQUITY PORTFOLIO,,0000950123-11-054817.txt,716716,20110527,MORGAN STANLEY VARIABLE INVESTMENT SERIES,20110331,https://www.sec.gov/Archives/edgar/data/716716...,,,,yes
13,573000,1782671,ENN Energy Holdings Ltd.,(c),EUROPEAN EQUITY PORTFOLIO,EUROPEAN EQUITY PORTFOLIO,,0000950123-11-054817.txt,716716,20110527,MORGAN STANLEY VARIABLE INVESTMENT SERIES,20110331,https://www.sec.gov/Archives/edgar/data/716716...,,,,yes
14,226000,146724,Sichuan Expressway Co. Ltd.,(c),EUROPEAN EQUITY PORTFOLIO,EUROPEAN EQUITY PORTFOLIO,,0000950123-11-054817.txt,716716,20110527,MORGAN STANLEY VARIABLE INVESTMENT SERIES,20110331,https://www.sec.gov/Archives/edgar/data/716716...,,,,yes
